In [ ]:
import pickle
import boris_extraction as boris
import multirecording_spikeanalysis as spike
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import sem
from itertools import combinations
def hex_2_rgb(hex_color): # Orange color
    rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
    return rgb_color

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))

## Wilcoxon Single Cell Analysis

In [1]:
def get_fam_score(acquisition_vs_novel, recall_vs_cagemate, recall_vs_novel, cagemate_vs_novel, acquisition_vs_recall, id_score):
    fam_score = 0
    if acquisition_vs_novel == 'not significant':
        if recall_vs_cagemate != 'not significant':
            fam_score +=1
        if recall_vs_novel != 'not significant':
            fam_score +=1
        if cagemate_vs_novel != 'not significant':
            fam_score +=2
        if (id_score != 0):
            if id_score == 'familiarity?':
                fam_score = fam_score
            else:
                fam_score += (-2)
        if acquisition_vs_recall != 'not significant':
            fam_score += 1
    return fam_score

def get_id_score(acquisition_vs_novel, recall_vs_cagemate, recall_vs_novel, cagemate_vs_novel, acquisition_vs_recall):
    id_score = 0
    if acquisition_vs_recall == 'not significant':
        if (recall_vs_cagemate != 'not significant') & (cagemate_vs_novel != 'not significant'):
                id_score = 'cagemate'
        if (recall_vs_novel != 'not significant') & (recall_vs_cagemate != 'not significant'):
                id_score = 'familiar'
        if (recall_vs_novel != 'not significant') & (cagemate_vs_novel != 'not significant'):
                id_score = 'novel'
        if (recall_vs_novel != 'not significant') & (cagemate_vs_novel != 'not significant') & (recall_vs_cagemate != 'not significant'):
             id_score = 'familiarity?'
    if acquisition_vs_recall != 'not significant':
         if (recall_vs_novel != 'not significant') & (recall_vs_cagemate != 'not significant'):
                id_score = 'recall'
    return id_score


In [ ]:
           #offset included                    offset excluded 
# 2, 5, -2 =      80                                 72
# 1 , 5, -2 =     65                                  65
# 0, 6, -1 =      65                                       
# 1, 5, -1 =      64                                  58
# 2, 4, 1 =       62
# 3, 5, 0 =       62
# 3, 5, -1 =      61
# 3, 5, -2 =      61                                  
# 2, 4, -1 =      71                                  73
# 2, 4, 0 =       59         
# 3, 4, 0 =       59

In [2]:
equalize = 2
baseline_window = 2
offset = 0
print(pca_analysis.fishers_exact_wilcox('novel', 'cagemate', equalize = equalize, baseline_window=baseline_window,offset= offset, event3 = None))
pca_analysis.fishers_exact_wilcox('familiar', 'acquisition',equalize = equalize, baseline_window=baseline_window,offset= offset, event3 = None)

NameError: name 'pca_analysis' is not defined

In [ ]:
df_list = []
for comparison in ['2s novel vs 2s baseline', '2s cagemate vs 2s baseline', '2s familiar vs 2s baseline', '2s acquisition vs 2s baseline']:
    df = phase3_collection.wilcox_dfs[comparison]
    df_list.append(df)

is_first = True 
for df in df_list:
    if is_first:
        master_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event','event1 vs event2']]
        is_first = False
    else: 
        temp_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event', 'event1 vs event2']]
        master_df = master_df.merge(temp_df,  on=['Subject', 'Recording', 'original unit id'], how = 'left')
    
#novel vs baseline

In [ ]:
master_df.columns = ['Subject',
 'Recording',
 'original unit id',
 'Event_x',
 'novel vs baseline',
 'Event_y',
 'cagemate vs baseline',
 'Event_x',
 'familiar vs baseline',
 'Event_y',
 'acquisition vs baseline' ]
master_df = master_df[['original unit id',
 'novel vs baseline',
 'cagemate vs baseline',
 'familiar vs baseline',
 'acquisition vs baseline'
 ]]

In [ ]:
def group_cells(novel, cagemate, fam, acquisition):
    group = ''
    if novel != 'not significant':
        group = 'novel'
    if cagemate != 'not significant':
        group = group + 'cagemate' 
    if fam != 'not significant':
        group = group + 'fam'
    if acquisition != 'not significant':
        group = group + 'acquisition'
    return group

master_df['cell group'] = master_df.apply(lambda row: group_cells(row['novel vs baseline'],
                                                                  row['cagemate vs baseline'],
                                                                  row['familiar vs baseline'],
                                                                  row['acquisition vs baseline']), axis = 1)
results = master_df.groupby('cell group').count()
results